# L1000 to RNA-seq conversion pipeline - Training, Predicting & Evaluating

This is an L1000 to RNA-seq conversion pipeline. The pipeline takes 978-dimensional Level3 L1000 profiles as input and returns 25,312-dimensional RNA-seq like profiles. A cycleGAN model in step 1 converts gene expression values in L1000 to those in RNA-seq only for landmark genes. Then, step 2 takes the output profiles of step 1 and extrapolates the profiles to the 25,312 full genome profiles.



In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from numpy.random import seed
import umap
from sklearn.manifold import TSNE
import time

randomState = 123
seed(randomState)

Parameters

In [ ]:
step1_exp_index = 30
step2_exp_index = 30
num_samples = 50000

step1_y_true_filename = "y_true_L1000_MCF7.txt"
step1_y_pred_filename = "y_pred_L1000_MCF7.txt"
step2_y_true_filename = "y_true_ARCHS4_MCF7.txt"
step2_y_pred_filename = "y_pred_ARCHS4_MCF7.txt"
eval_dataset_nameA = "L1000_MCF7"
eval_dataset_nameB = "ARCHS4_MCF7_landmark"

# step1_y_true_filename = "y_true_L1000_GTEx.txt"
# step1_y_pred_filename = "y_pred_L1000_GTEx.txt"
# step2_y_true_filename = "y_true_ARCHS4_GTEx.txt"
# step2_y_pred_filename = "y_pred_ARCHS4_GTEx.txt"
# eval_dataset_nameA = "GTEx_L1000"
# eval_dataset_nameB = "GTEx_RNAseq_landmark"
# eval_output_dataset_name = "GTEx_RNAseq"




## Training: Step 1

In [ ]:
!python functions/delete.py --exp_index $step1_exp_index

In [ ]:
!python functions/cyclegan_transcript.py --dataset_nameA "L1000" --dataset_nameB "ARCHS4" --n_epochs 100 --decay_epoch 50 --input_dimA 962 --hidden_dimA 512 --output_dimA 128 --input_dimB 962 --hidden_dimB 512 --output_dimB 128 --num_samples $num_samples --batch_size 100 --exp_index $step1_exp_index --prediction_folder "../output/"$step1_exp_index"/prediction/" --lambda_id 0.0 --benchmark_evaluation --eval_dataset_nameA $eval_dataset_nameA --eval_dataset_nameB $eval_dataset_nameB 

## Training: Step 2

In [ ]:
!python functions/delete.py --exp_index $step2_exp_index --step2

In [ ]:
!python functions/extrapolation_transcript.py --input_dataset_name "ARCHS4_50000_input" --output_dataset_name "ARCHS4_50000_output" --n_epochs 100 --decay_epoch 10 --input_dim 962 --hidden_dim 2048 4096 8192 --output_dim 23614 --num_samples $num_samples --batch_size 100 --exp_index $step2_exp_index --valid_ratio 0.01 --test_ratio 0.01 --y_pred_output_filename "y_pred.txt" --y_true_output_filename "y_true.txt" --early_stopping --early_stopping_epoch 3 --early_stopping_tol 0.0001 --prediction_folder ../output_step2/$step2_exp_index/prediction
# 

## Predicting: Step 0 Preprocessing input file (Optional)

978 landmark genes in GCTX -> 962 landmark genes in feather format)

In [ ]:
!python functions/preprocessing_input_data.py --input_filename ../data/LINCS_CFDE/L1000_GSE92742_landmark_only/L1000_GSE92742_1.gctx --output_filename ../data/LINCS_CFDE/L1000_GSE92742_landmark_only_feather/L1000_GSE92742_1.gctx

## Predicting: Step 1 Running cycleGAN (L1000->RNA-seq)

Input file format: feather
Output file format: txt (tab-separated)

In [ ]:
!python functions/cyclegan_transcript.py --ispredicting --exp_index $step1_exp_index --load_model_index $step1_model_index --eval_dataset_nameA ../data/Evaluation/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n203x962_celllineMCF7.f --y_pred_output_filename step1.txt --prediction_folder "../prediction/" 

## Predicting: Step 2 Extrapolating (962 dim RNA-seq -> 25,312 dim RNA-seq)

In [ ]:
!python functions/extrapolation_transcript.py --ispredicting --exp_index $step2_exp_index --eval_input_dataset_name ../prediction/step1.txt --y_pred_output_filename step2.txt --prediction_folder "../prediction/"

## Evaluation

In [ ]:
!python functions/evaluation.py --y_true ../data/Evaluation/ARCHS4_human_matrix_v9_n203x25312_celllineMCF7.f --y_pred ../prediction/step2.txt